In [410]:
import pandas as pd
import numpy as np

In [411]:
data_df = pd.read_csv('Group_2_marco_statistics.txt', sep='\t')

## Quick look at this unruly dataset (get ready, it’s a mess!)


In [412]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3603 entries, 0 to 3602
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Area Code        3592 non-null   object
 1   Area Code (M49)  3594 non-null   object
 2   Area             3600 non-null   object
 3   Item Code        3594 non-null   object
 4   Item             3602 non-null   object
 5   Element Code     3592 non-null   object
 6   Element          3598 non-null   object
 7   Unit             3485 non-null   object
 8   Y2019            3246 non-null   object
 9   Y2020            3227 non-null   object
 10  Y2021            3014 non-null   object
 11  Y2022            2441 non-null   object
 12  Y2023            1531 non-null   object
dtypes: object(13)
memory usage: 366.1+ KB


In [413]:
data_df.head()

,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Unit,Y2019,Y2020,Y2021,Y2022,Y2023
0,3,'008,Albania,22008,Gross Domestic Product,6224,Value Standard Local Currency,million SLC,1691903.429768,1647431.1,1856172.3,2138339.2,2257544.743428
1,3,'008,Albania,22008,Gross Domestic Product,6110,Value US$,million USD,15401.82608,15162.734468,17930.56704,18916.380686,22430.768974
2,3,'008,Albania,22008,Gross Domestic Product,6119,Value US$ per capita,USD,5359.239078,5288.989573,6281.046775,6655.258391,7919.241676
3,3,'008,Albania,22008,Gross Domestic Product,6225,"Value Standard Local Currency, 2015 prices",million SLC,1633423.912176,1576554.211752,1720197.512823,1803737.613446,1844321.709748
4,3,'008,Albania,22008,Gross Domestic Product,6185,"Value US$ per capita, 2015 prices",USD,4512.262019,4365.847635,4783.885801,5038.076094,5169.405465


#### 1. Replacing ‘n/a’ and ‘9999999’ popping up where they shouldn’t. Let’s turn them into NaN.

In [414]:
data_df.isin(['n/a', '9999999']).sum()

Area Code          5
Area Code (M49)    0
Area               0
Item Code          1
Item               0
Element Code       1
Element            0
Unit               0
Y2019              3
Y2020              3
Y2021              3
Y2022              0
Y2023              2
dtype: int64

In [415]:
data_df.replace(['n/a', '9999999'], np.nan, inplace=True)

#### 2. Drop Rows that have NAN values greter than 4.

In [416]:
(data_df.isnull().sum(axis=1) > 4).sum()


np.int64(340)

In [417]:
data_df = data_df.dropna(thresh=len(data_df.columns) - 4)
(data_df.isnull().sum(axis=1) > 4).sum()

np.int64(0)

#### 3. Making Numbers Actually... Numbers

In [418]:
columns_names = ['Area Code (M49)', 'Item Code', 'Element Code', 'Y2019', 'Y2020', 'Y2021', 'Y2022', 'Y2023']
print(data_df[columns_names].dtypes)

Area Code (M49)    object
Item Code          object
Element Code       object
Y2019              object
Y2020              object
Y2021              object
Y2022              object
Y2023              object
dtype: object


In [419]:
data_df[columns_names] = data_df[columns_names].apply(pd.to_numeric, errors='coerce')

In [420]:
print(data_df[columns_names].dtypes)

Area Code (M49)    float64
Item Code          float64
Element Code       float64
Y2019              float64
Y2020              float64
Y2021              float64
Y2022              float64
Y2023              float64
dtype: object


#### 4. Get rid of bad item codes like -9999999 .

In [421]:
data_df[data_df['Item Code'] < 0]


,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Unit,Y2019,Y2020,Y2021,Y2022,Y2023
17,3,NaN,Albania,-9999999.0,Gross Fixed Capital Formation,6187.0,"Share of GDP US$, 2015 prices",%,22.979889,23.480017,25.654031,26.21078,26.282782
322,57,NaN,Belarus,-9999999.0,Value Added (Agriculture),61550.0,Annual growth Standard Local Currency,%,12.758714,13.355582,8.721305,NaN,NaN
1194,79,NaN,Germany,-9999999.0,Gross Output (Agriculture),6110.0,Value US$,million USD,64115.351677,61671.460696,NaN,NaN,NaN


In [422]:
data_df = data_df[data_df['Item Code'] > 0]
print(data_df[data_df['Item Code'] < 0])


Empty DataFrame
Columns: [Area Code, Area Code (M49), Area, Item Code, Item, Element Code, Element, Unit, Y2019, Y2020, Y2021, Y2022, Y2023]
Index: []


#### 5. Handling outliers by capping them at the mean

In [423]:
for year in ['Y2019', 'Y2020', 'Y2021', 'Y2022', 'Y2023']:
    print(data_df[year][data_df[year] > data_df[year].mean() + 10 * data_df[year].std()])

1272    4.767419e+07
1275    4.116913e+07
1322    4.641581e+07
2692    1.096083e+08
2695    8.906355e+07
2782    1.061435e+08
Name: Y2019, dtype: float64
1272    4.842542e+07
1275    3.930189e+07
1322    4.726872e+07
2692    1.076581e+08
2695    8.670009e+07
2782    1.051341e+08
Name: Y2020, dtype: float64
1272    5.519893e+07
1322    5.344323e+07
2692    1.352950e+08
2695    9.156771e+07
2782    1.321267e+08
Name: Y2021, dtype: float64
1272    6.607519e+07
1322    6.402630e+07
2692    1.534352e+08
2695    8.967253e+07
2782    1.504418e+08
Name: Y2022, dtype: float64
1272    7.201945e+07
2692    1.560102e+08
2695    9.031010e+07
Name: Y2023, dtype: float64


In [424]:
for year in ['Y2019', 'Y2020', 'Y2021', 'Y2022', 'Y2023']:
    mean, std = data_df[year].mean(), data_df[year].std()
    data_df[year] = np.where(data_df[year] > mean + 10 * std, mean, data_df[year])

#### 6. Standardizing the unit names


In [425]:
data_df['Unit'].unique()

array(['million SLC', 'million USD', 'USD', '%', nan, '3.029878',
       'million SLCz', '2022-05-01', '183', '%z', '6.049564'],
      dtype=object)

In [426]:
unit_replacements = {'million SLC': 'M SLC', 'million USD': 'M USD', 'USD': 'USD'}
data_df['Unit'] = data_df['Unit'].replace(unit_replacements)
data_df['Unit'].unique()

array(['M SLC', 'M USD', 'USD', '%', nan, '3.029878', 'million SLCz',
       '2022-05-01', '183', '%z', '6.049564'], dtype=object)

#### 7. Handling percent signs

In [427]:

data_df.dropna(inplace=True)
for col in data_df.columns:
    if data_df[col].dtype == 'object' and data_df[col].str.contains('%').any():
        print(data_df[col][data_df[col].str.contains('%')])
        data_df[col] = data_df[col].str.replace('%', '')

1145    %
Name: Unit, dtype: object


#### 8. Removing quote marks from Area Code

In [428]:
data_df.head()

,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Unit,Y2019,Y2020,Y2021,Y2022,Y2023
1145,'276,79.0,Germany,22015.0,Gross Fixed Capital Formation,6182.0,"Annual growth US$, 2015 prices",,1.725703,-2.44643,-0.155592,0.119168,2.131398


In [429]:
data_df['Area Code'] = data_df['Area Code'].str.replace("'", "").astype(float)
data_df.head()

,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Unit,Y2019,Y2020,Y2021,Y2022,Y2023
1145,276.0,79.0,Germany,22015.0,Gross Fixed Capital Formation,6182.0,"Annual growth US$, 2015 prices",,1.725703,-2.44643,-0.155592,0.119168,2.131398


#### 9. Setting a Friendly Index

In [430]:
data_df.head()

,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Unit,Y2019,Y2020,Y2021,Y2022,Y2023
1145,276.0,79.0,Germany,22015.0,Gross Fixed Capital Formation,6182.0,"Annual growth US$, 2015 prices",,1.725703,-2.44643,-0.155592,0.119168,2.131398


In [431]:
data_df.set_index(['Area Code', 'Item Code'], inplace=True)

#### 10. Renaming columns to human-friendly names :"

In [432]:
data_df.columns

Index(['Area Code (M49)', 'Area', 'Item', 'Element Code', 'Element', 'Unit',
       'Y2019', 'Y2020', 'Y2021', 'Y2022', 'Y2023'],
      dtype='object')

In [433]:
data_df.rename(columns=lambda x: x.replace('Y', 'Year_'), inplace=True)
data_df.columns

Index(['Area Code (M49)', 'Area', 'Item', 'Element Code', 'Element', 'Unit',
       'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023'],
      dtype='object')